In [ ]:
pip install contractions


In [ ]:
import boto3, sys,sagemaker                             
import pandas as pd   
import pandas as pd
import numpy as np
import nltk
import string
import contractions
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
# plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket_name = 'howardvip-clinical-data'
def table(table_name, nrows=0):
    data_path = 's3://{}/{}'.format(bucket_name, 'NOTEEVENTS.csv.gz')
    if not nrows:
        return pd.read_csv(data_path)
    return pd.read_csv(data_path, nrows=nrows)


In [ ]:
df = table('NOTEEVENTS.csv.gz',nrows=5000)

In [ ]:
df

In [ ]:
dfText = df.loc[:,["TEXT"]]


In [ ]:
dfText["TEXT"][2]

In [ ]:
dfText['no_contract'] = dfText['TEXT'].apply(lambda x: [contractions.fix(word) for word in x.split()])
dfText.head()

In [ ]:
dfText["TEXT"] = [' '.join(map(str, l)) for l in  dfText['no_contract']]
dfText.head()

In [ ]:
nltk.download('punkt')
dfText['tokenized'] = dfText['TEXT'].apply(word_tokenize)


In [ ]:
dfText

In [ ]:
dfText['TEXT'][2]

In [ ]:
dfText['lower'] =dfText['tokenized'].apply(lambda x: [word.lower() for word in x])
dfText.head()

In [ ]:
punc = string.punctuation
dfText['no_punc'] = dfText['lower'].apply(lambda x: [word for word in x if word not in punc and word.isalpha()])
dfText.head()

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
dfText['stopwords_removed'] = dfText['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
dfText.head()

In [ ]:
nltk.download('averaged_perceptron_tagger')
dfText['pos_tags'] = dfText['stopwords_removed'].apply(nltk.tag.pos_tag)
dfText.head()

In [ ]:
nltk.download('wordnet')
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
dfText['wordnet_pos'] = dfText['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
dfText.head()

In [ ]:
wnl = WordNetLemmatizer()
dfText['lemmatized'] = dfText['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
dfText.head()

In [ ]:
dfText.to_csv('TEXT_clean.csv')

In [ ]:
dfText.to_csv('cleanedText.csv')

In [ ]:
dfText

In [ ]:
dfText['lemmatized'][1]

In [ ]:
dfText['TEXT'][1]

In [ ]:
pip install gensim

In [ ]:
import logging
from gensim.models.word2vec import Word2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
w2v_model = Word2Vec(dfText['lemmatized'], vector_size=100, min_count=2)


In [ ]:
w2v_model.wv.most_similar(['cancer'])

In [ ]:
w2v_model.wv.most_similar(['doctor'])

In [ ]:
w2v_model.wv.most_similar(['patient'])

In [ ]:
w2v_model.wv.most_similar(['disease'])

In [ ]:
w2v_model.save('modelwv')

In [ ]:
new_model = Word2Vec.load('modelwv')

In [ ]:
new_model.wv.most_similar(['disease'])

In [ ]:
dfText['tokenizedSent'] = dfText['TEXT'].apply(sent_tokenize)

In [ ]:
dfText['lower1'] =dfText['tokenizedSent'].apply(lambda x: [[word.lower() for word in sent] for sent in x])

In [ ]:
punc = string.punctuation
dfText['no_punc1'] = dfText['lower1'].apply(lambda x: [[word for word in sent if word not in punc and word.isalpha()] for sent in x])

In [ ]:
dfText['stopwords_removed1'] = dfText['no_punc1'].apply(lambda x: [[word for word in sent if word not in stop_words]for sent in x])

In [ ]:
dfText['pos_tags1'] = dfText['stopwords_removed1'].apply(lambda x: [[nltk.tag.pos_tag(sent)]for sent in x])

In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
dfText['wordnet_pos1'] = dfText['pos_tags1'].apply(lambda x: [[(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in sent]for sent in x])

In [ ]:
dfText['lemmatized1'] = dfText['wordnet_pos1'].apply(lambda x: [[wnl.lemmatize(word, tag) for word, tag in sent] for sent in x])
dfText.head()